In [18]:
import requests
import pandas as pd
import numpy as np
import xarray as xr
import sqlite3
import os
from datetime import datetime
import cdsapi
import csv

In [19]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table : ", error)

# Connexion à la bdd piezo
Construction d'un dataframe 'df_piezo' à partir de la requête sur la bdd

In [20]:
region = "OCC"

In [21]:
connexion = sqlite3.connect('./data/liste_piezos.db')
curseur = connexion.cursor()
requete = f"""
    SELECT code_bss, date_debut, date_fin, nb_mesures_piezo, latitude, longitude 
        FROM data_piezo
    INNER JOIN code_region
    ON code_dpt = departement
    WHERE code_region IS '{region}'
    AND date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""

df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])

In [22]:
df_piezo

,date_debut,date_fin,nb_mesures_piezo,latitude,longitude
code_bss,,,,,
09654X0569_CNRP20,2004-03-14,2022-05-24,6243,43.848274,4.607080
08592X0175_FOR170,2008-11-17,2020-12-31,618,44.557278,2.251595
09892X0679_EXH70,2004-02-01,2022-05-24,6544,43.695576,3.327317
10615X0047_F0,2009-02-27,2022-04-07,4408,43.054528,2.921968
09648X0083_F90-1,1995-01-31,2022-05-02,7659,43.744145,4.230897
...,...,...,...,...,...
08576X0022_HY,2009-04-08,2019-06-04,3398,44.506078,1.654776
08577X0029_HY,2009-04-08,2021-01-04,4291,44.521516,1.668573
09635X0003_FHY,1997-06-01,2022-05-24,8323,43.759836,3.624061


# Fonction pour lancer les requêtes

In [23]:
def get_on_cds_api(dataframe_piezo, piezo, path):
    # Initialisation de la connexion à l'API CDS
    # wait_until_complete = False : le programme Python n'attend pas que l'API réponde (issu de la libraie asyncio)
    # delete = False : le fichier créé sur le serveur source n'est pas supprimé automatiquement 
    
    date_debut = dataframe_piezo.loc[piezo,'date_debut'].year
    date_fin = dataframe_piezo.loc[piezo,'date_fin'].year
    
    # La limite du nombre de données est de 120000. Avec 10 variables, 31 jours, et 12 mois, on n epeut récupérer que 32 années.
    # Si l'interval d'année est supérieur à 32, on réduit la plage de temps à date de fin - 31 ans pour avoir 32 années max.
    if date_fin - date_debut >= 32:
        date_debut = date_fin - 31
    
    c = cdsapi.Client(wait_until_complete=False, delete=False)
    
    # Données à récupérer auprès de l'API
    r = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '2m_temperature', 'convective_precipitation', 'convective_rain_rate',
                    'convective_snowfall', 'large_scale_rain_rate', 'large_scale_snowfall', 'snowfall',
                    'large_scale_snowfall_rate_water_equivalent', 'potential_evaporation',
                    'total_precipitation',
                ],
                'year': [str(date) for date in (np.arange(date_debut, date_fin))],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': '00:00',
                'area': [
                    df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"], df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"],
                ],
            })    
    return r

## Lancer les requêtes auprès de l'API
!!! NE LANCER QU'UNE SEULE FOIS POUR NE PAS CREER DES DOUBLONS DE REQUETES !!!

Un dictionnaire est créé avec en clé le code_bss du piezo et en valeur l'id de la requête

Ne pas lancer la cellule ci-dessous si des requetes sont en cours. 

In [24]:
# Parcourir les codes_bss des piezos dans la liste précédemment créé :
dict_bss = {}

In [25]:
go_requete = input("""
            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
""")
                   
if go_requete == 'req' :
                      
    for piezo in df_piezo.index:
        path_piezo = 'data/era5/' + piezo
        path_nc = f"./{path_piezo}.nc"
        path_pkl = f"{path_piezo}.pkl"

        if os.path.isfile(path_nc) : #si le fichier nc existe on le converti en pkl
            print(f"Le fichier {path_nc} existe.")
            df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
            print(f"Fichier {piezo}.nc converti en fichier pickle")
            os.remove(path_nc)
            print(f"Fichier {path_nc} supprimé")

        elif os.path.isfile(path_pkl) : #si le fichier pkl existe, on affiche qu'il existe
            print(f"Le fichier {piezo}.pkl existe déjà.")
            # puis on le supprime de la liste
            df_piezo.drop(index=piezo, axis=1)

        else : # Sinon on lance une requête auprès de l'API
            result = get_on_cds_api(df_piezo, piezo, path_nc)
            # récupération de l'id de la requête
            dict_bss[piezo] = result.reply["request_id"]
            print(f"{datetime.now()} : chargement du fichier {piezo}.nc en cours ({dict_bss[piezo]}).")
            
            # Enregistrement de l'id de requete dans un fichier csv
            if os.path.isfile('./data/hist_requetes.csv'): # Si le fichier existe                
                with open('./data/hist_requetes.csv', 'a', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
            else : # Si le fichier n'existe pas
                with open('./data/hist_requetes.csv', 'w', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    writer.writerow(["code_bss", "id_requete"])
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
else : 
    print("Aucune requête n'a été lancée.")


            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
req


2022-07-21 13:33:45,391 INFO Welcome to the CDS
2022-07-21 13:33:45,391 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,468 INFO Welcome to the CDS
2022-07-21 13:33:45,469 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,530 INFO Welcome to the CDS
2022-07-21 13:33:45,531 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,591 INFO Welcome to the CDS
2022-07-21 13:33:45,593 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:33:45.431814 : chargement du fichier 09654X0569_CNRP20.nc en cours (14e7b9c4-eae1-401c-ad9c-46363f9a7347).
2022-07-21 13:33:45.502896 : chargement du fichier 08592X0175_FOR170.nc en cours (8c0fdca8-d253-4caa-b7e5-9968f12913c0).
2022-07-21 13:33:45.563774 : chargement du fichier 09892X0679_EXH70.nc en cours (d9b727dc-2adb-441b-9b39-7b2768d8616c).


2022-07-21 13:33:45,677 INFO Welcome to the CDS
2022-07-21 13:33:45,679 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,740 INFO Welcome to the CDS
2022-07-21 13:33:45,742 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:33:45.651273 : chargement du fichier 10615X0047_F0.nc en cours (236db60c-c39b-4415-96bf-a105f6a832e3).
2022-07-21 13:33:45.712718 : chargement du fichier 09648X0083_F90-1.nc en cours (30f5a1d4-c96c-47e9-8eea-a3bf82fa6ef6).
2022-07-21 13:33:45.779320 : chargement du fichier 10906X0039_C2-1.nc en cours (6020aefe-70e0-496c-85ff-77cc5be4a0c9).


...

2022-07-21 13:34:06.136964 : chargement du fichier 10882X0215_ADOUX.nc en cours (0fbe163b-302a-4522-970b-ba4bdf9f839f).
2022-07-21 13:34:06.254614 : chargement du fichier 09625X0231_LADOUX.nc en cours (24fe87a3-85d3-49f4-837e-8ce4fbd41b88).
2022-07-21 13:34:06.316984 : chargement du fichier 09126X0275_RABY.nc en cours (4a2ab5bd-d692-4ed0-b773-9d7f34422098).


2022-07-21 13:34:06,345 INFO Welcome to the CDS
2022-07-21 13:34:06,346 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:34:06,411 INFO Welcome to the CDS
2022-07-21 13:34:06,411 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:34:06,472 INFO Welcome to the CDS
2022-07-21 13:34:06,473 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:34:06,534 INFO Welcome to the CDS
2022-07-21 13:34:06,534 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:34:06.383898 : chargement du fichier 08576X0022_HY.nc en cours (f8d9c845-f14f-4581-b019-8317ef8efcc4).
2022-07-21 13:34:06.444614 : chargement du fichier 08577X0029_HY.nc en cours (bf1e711b-7018-4e40-b203-0568fe291213).
2022-07-21 13:34:06.505687 : chargement du fichier 09635X0003_FHY.nc en cours (e4a81d22-7edf-4ce5-839a-5fef319a6cca).
2022-07-21 13:34:06.566919 : chargement du fichier 10165X0021_CAUVY.nc en cours (980333d9-614c-43f6-a9ac-6758112e1536).


2022-07-21 13:34:06,595 INFO Welcome to the CDS
2022-07-21 13:34:06,595 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:34:06.627972 : chargement du fichier 10581X0201_SAGRAZ.nc en cours (6571df72-4bc2-4af1-9742-d8cda20df656).


## Récupérer automatiquement les requêtes abouties 
Peut être lancé à tout moment pour récupérer des fichiers téléchargés 

In [15]:
# Chargement du dictionnaire à partir du csv d'historiques de requetes
with open('./data/hist_requetes.csv', mode='r') as f:
    reader = csv.reader(f, delimiter=";")
    # Permet de sortir la ligne d'en-tête du dictionnaire
    headings = next(reader)
    dict_bss_from_csv = {rows[0]:rows[1] for rows in reader}

nbre_fichier_OK = 0

    
for piezo in dict_bss_from_csv.copy().keys():
    path_piezo = 'data/era5/' + piezo
    path_nc = f"./{path_piezo}.nc"
    path_pkl = f"{path_piezo}.pkl"

    if not os.path.isfile(path_nc) :
        try:
            reply = dict(request_id=dict_bss_from_csv[piezo]) # request_id from above
            new_client = cdsapi.Client(wait_until_complete=False, delete=False)
            result = cdsapi.api.Result(new_client, reply)
            result.update()
            reply = result.reply
            if reply['state'] == 'completed':
                nbre_fichier_OK += 1
                result.download(path_nc)
                print(f"{datetime.now()} : Fichier {piezo}.nc téléchargé.")
                df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
                print(f"Fichier {piezo}.nc converti en fichier pickle")
                os.remove(path_nc)
                print(f"Fichier {path_nc}.nc supprimé")
                # Si le fichier a été téléchargé on le supprime de la liste du dictionnaire
                del dict_bss[piezo]
        except:
            #print(f"Récupération de l'ID {dict_bss_from_csv[piezo]} impossible.")
            pass
if nbre_fichier_OK == 0:
    print("Aucun fichier de récupéré.")
else :
    print(f"{nbre_fichier_OK} fichiers ont été téléchargés.")

# Mise à jour de la base de donnéees (champ fichier_ERA5)
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)
print("Base de données à jour")

2022-07-21 13:25:56,423 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data4/adaptor.mars.internal-1658141695.9897885-3825-17-5ada63eb-54f3-4bc6-b561-9670a4ccbd07.nc to ./data/era5/08276X0085_F1.nc (268.8K)
2022-07-21 13:25:56,731 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data9/adaptor.mars.internal-1658141696.615006-16659-15-cc4467ae-b828-46e2-af82-477bcbf02da8.nc to ./data/era5/08511X0036_F1.nc (268.8K)
2022-07-21 13:25:56,979 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data0/adaptor.mars.internal-1658141696.345232-12802-12-b7b2241f-d91a-4ec6-a53f-9543b354daac.nc to ./data/era5/09783X0018_F1.nc (268.8K)
2022-07-21 13:25:57,150 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data9/adaptor.mars.internal-1658144568.333093-25988-4-bee70d6d-0c84-47ad-82c5-d9a86d38b9ac.nc to ./data/era5/09008X0028_F1.nc (1

2022-07-21 13:26:02,609 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data7/adaptor.mars.internal-1658150979.0544522-12507-12-1f12d5b6-93da-48f9-8d0c-93044ea2930a.nc to ./data/era5/07131X0019_P1.nc (131.9K)
2022-07-21 13:26:02,825 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data1/adaptor.mars.internal-1658150972.1095147-3548-10-fd84fc59-0e31-4895-9e8f-ca282aff07bf.nc to ./data/era5/06432X0059_P1.nc (149K)
2022-07-21 13:26:02,994 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data9/adaptor.mars.internal-1658153086.2906601-26933-8-6e32a38a-c832-42a6-9d3c-438cb3ef00b1.nc to ./data/era5/07786X0047_BP1.nc (157.5K)
2022-07-21 13:26:03,157 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data6/adaptor.mars.internal-1658155784.8220224-7874-1-04d317cf-8e87-4f2b-b731-9e71eb81525e.nc to ./data/era5/08977X0009_F4NP1.nc (268.

2022-07-21 13:26:08,513 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data3/adaptor.mars.internal-1658165194.511573-26784-2-9f4ec439-3909-4022-a0e8-5d87f9b0c16a.nc to ./data/era5/06882X0214_P2.nc (149K)
2022-07-21 13:26:08,655 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data0/adaptor.mars.internal-1658171061.236412-19555-15-d3828661-6bec-455e-a52a-954a39a2dec3.nc to ./data/era5/06417X0015_P2.nc (123.3K)
2022-07-21 13:26:08,856 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data8/adaptor.mars.internal-1658169111.7604299-7364-19-50fdab8c-f5d6-4412-a1dd-ceb815b58098.nc to ./data/era5/08056X0026_P2.nc (174.6K)
2022-07-21 13:26:09,024 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data4/adaptor.mars.internal-1658167278.2677433-32699-15-0f4a7f74-4c2f-4a2a-8d2f-e25975cb3d15.nc to ./data/era5/07143X0018_P2.nc (123.3K)


2022-07-21 13:26:15,985 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data1/adaptor.mars.internal-1658190413.1167676-5760-3-5de02373-4ea8-4373-8287-13feaadce3b0.nc to ./data/era5/07545X0024_F3.nc (268.8K)
2022-07-21 13:26:16,157 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data7/adaptor.mars.internal-1658183316.437986-15129-9-50bddda7-375e-4d08-9282-789fa7d850a4.nc to ./data/era5/10306X0034_F3.nc (268.8K)
2022-07-21 13:26:16,334 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data2/adaptor.mars.internal-1658182776.4407237-22349-13-03321687-9558-4dc2-82db-276964b46efb.nc to ./data/era5/07595X0006_F3.nc (226K)
2022-07-21 13:26:16,482 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data2/adaptor.mars.internal-1658190403.7944155-4694-4-cbe3d75c-2688-45d0-b715-8bd3c6c91955.nc to ./data/era5/09782X0016_F3.nc (268.8K)
20

2022-07-21 13:26:18.409722 : Fichier 07804X0538_F3.nc téléchargé.
Fichier 07804X0538_F3.nc converti en fichier pickle
Fichier ./data/era5/07804X0538_F3.nc.nc supprimé


...

2022-07-21 13:27:55.805323 : Fichier 00853X0018_S1.nc téléchargé.
Fichier 00853X0018_S1.nc converti en fichier pickle
Fichier ./data/era5/00853X0018_S1.nc.nc supprimé


2022-07-21 13:27:56,021 INFO Download rate 1.7M/s
2022-07-21 13:27:56,068 INFO Downloading https://download-0018.copernicus-climate.eu/cache-compute-0018/cache/data7/adaptor.mars.internal-1658394684.498041-20430-11-34fb6d22-c6ee-4a46-81a2-ea476b4ef8cc.nc to ./data/era5/02617X0009_S1.nc (268.8K)


2022-07-21 13:27:56.021772 : Fichier 02636X0009_S1.nc téléchargé.
Fichier 02636X0009_S1.nc converti en fichier pickle
Fichier ./data/era5/02636X0009_S1.nc.nc supprimé


2022-07-21 13:27:56,255 INFO Download rate 1.4M/s


2022-07-21 13:27:56.255947 : Fichier 02617X0009_S1.nc téléchargé.
Fichier 02617X0009_S1.nc converti en fichier pickle
Fichier ./data/era5/02617X0009_S1.nc.nc supprimé
547 fichiers ont été téléchargés.
Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


## Jointure era5 et piezo

In [17]:
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")
piezos_communs = np.intersect1d(bss_era5, bss_piezo)

path = "./data/combined_data/"
# Le fichier doit avoir un minimum de 100 mesures
nb_min_val_com = 100

for piezo in piezos_communs :
    if not os.path.isfile(f'{path}/comb_{piezo}.csv') : #si le fichier n'existe pas 
        df_piezo = pd.read_csv(f"./data/piezo/{piezo}.csv", sep = ";", index_col=0, parse_dates=True)
        # Group_by date_mesure dans le cas de plusieurs mesures le même jour ce qui provoquerait une erreur
        df_piezo = pd.DataFrame(df_piezo.groupby("date_mesure").mean())
        df_piezo = df_piezo.resample('1D').asfreq()
        df_era5 = pd.read_pickle(f"./data/era5/{piezo}.pkl").set_index('time').drop(["latitude", "longitude"], axis=1)
        if "expver" in df_era5.columns :
            df_era5 = df_era5.loc[df_era5["expver"]==1,:].drop("expver", axis=1)
        df = pd.merge(df_piezo, df_era5, how='inner', left_index=True, right_index=True)
        nb_com_obs = df.shape[0]
        if nb_com_obs >= nb_min_val_com :
            df.to_csv(f"data/combined_data/comb_{piezo}.csv" ,sep=";", index=True)
            print(f"Le fichier comb_{piezo}.csv a été créé et contient {nb_com_obs} obseravtions.")
        else :
            print(f"Le piezo {piezo} ne contient pas assez de données (seulement {nb_com_obs}).")
    else : 
        #print(f"Le fichier comb_{piezo}.csv existe déjà.")
        pass
        
# Mise à jour bdd
requete_fichier_combined = f"""
        UPDATE data_piezo
        SET fichier_combined =
        (CASE WHEN code_bss IN {tuple(piezos_communs)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_combined)
print("Base de données à jour")

Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


# Récupération manuelle

In [ ]:
reply = dict(request_id="54c52ee3-eee4-481d-b72e-fecd315085cb")
new_client = cdsapi.Client(wait_until_complete=False, delete=False)
result = cdsapi.api.Result(new_client, reply)
result.update()
reply = result.reply
if reply['state'] == 'completed':
    result.download('file.nc')

In [ ]:
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)

In [ ]:
update table set
    columnx = (case when condition then 25 else columnx end)